In [9]:
class argparser():
   device = "cuda:1"
   dataset = 'officehome'
   root_dir='/mnt/sda/zd/data/splitdata_new/'
   net='ViT-B/16'
   seed=0
   logdir=None
   device="cuda:1"
   n_clients=10
   n_experts=5
   inner_iter=10
   n_task=10
   # data setting parameter
   datapercent=8e-1
   batch=10
   # optimizer parameter
   lr=2e-3
   beta1=0.9
   beta2=0.98
   eps=1e-6
   weight_decay=0.2
import logging
def init():
    args = argparser()
    if args.logdir is not None:
        logging.basicConfig(filename=args.logdir, level=logging.DEBUG)
    else:
        logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
    args.random_state = np.random.RandomState(1)
    set_random_seed(args.seed)
    args = img_param_init(args)  # init the dataset parameters(domains and classnum)
    args.thresh = 1e-4
    return args

In [10]:
import sys
import os
import itertools

base_path = os.path.dirname(os.path.dirname(os.path.abspath(".")))
sys.path.append(base_path)

import torch.optim as optim
import numpy as np
from tqdm import tqdm
import logging

from utils.config import img_param_init, set_random_seed
from utils.dataload import DomainDataset, get_data
from model.clip import ClipModelMA, Adapter
from process import train_client, communicate, test_server, test_client, fetch

In [11]:

    
def init():
    args = argparser()
    if args.logdir is not None:
        logging.basicConfig(filename=args.logdir, level=logging.DEBUG)
    else:
        logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
    args.random_state = np.random.RandomState(1)
    set_random_seed(args.seed)
    args = img_param_init(args)  # init the dataset parameters(domains and classnum)
    args.thresh = 1e-4
    return args

In [12]:
def freeze_param(model):
    for name, param in model.named_parameters():
        param.requires_grad = False
args = init()
args.dataset = "officehome"
args.root_dir = "/mnt/sda/zd/data/splitdata_new/"
logging.info("Argument init successful!")
server_model: ClipModelMA = ClipModelMA(args.net, n_experts=args.n_experts, device=args.device)  # load the server data
dataloader: DomainDataset = get_data(args.dataset)(args, server_model.preprocess)
train_loaders, test_loaders, labels = dataloader.get_dataloader(0)
assert len(train_loaders) == args.n_clients, "This just mention you to make sure the n_clients \
                                            equals to the datasplit client number."
server_model.labels = labels
freeze_param(server_model.model)
logging.info("Data init successful!")
a = Adapter("ViT-B/16")
print(test_client(server_model, None,train_loaders[2],args.device))
for i in range(10):
    train_client(server_model, a, train_loaders[2], args.device, args)
    print(test_client(server_model, a,train_loaders[2],args.device))
# server_model.init_MoE() # expert number already saved in the server_model.
# logging.info("Model MoE init successful!")  # use the structure of single CLIP training server and client
# server_model.model.eval()
# server_model.model.to(args.device)



Argument init successful!
loading data in /mnt/sda/zd/data/splitdata_new/client0/task0/Art
loading data in /mnt/sda/zd/data/splitdata_new/client1/task0/Clipart
loading data in /mnt/sda/zd/data/splitdata_new/client2/task0/Clipart
loading data in /mnt/sda/zd/data/splitdata_new/client3/task0/Clipart
loading data in /mnt/sda/zd/data/splitdata_new/client4/task0/Clipart
loading data in /mnt/sda/zd/data/splitdata_new/client5/task0/Art
loading data in /mnt/sda/zd/data/splitdata_new/client6/task0/Product
loading data in /mnt/sda/zd/data/splitdata_new/client7/task0/Art
loading data in /mnt/sda/zd/data/splitdata_new/client8/task0/Product
loading data in /mnt/sda/zd/data/splitdata_new/client9/task0/Clipart
loading data in /mnt/sda/zd/data/splitdata_new/test/Clipart with ['Clipart', 'Product', 'Real World', 'Art']
loading data in /mnt/sda/zd/data/splitdata_new/test/Product with ['Clipart', 'Product', 'Real World', 'Art']
loading data in /mnt/sda/zd/data/splitdata_new/test/Real World with ['Clipart'

In [5]:
t = []
def get_feature():
    def hook(module, input, output):
        t.append(output)
    return hook
server_model.model.visual.transformer.resblocks[11].register_forward_hook(get_feature())

In [6]:
all_feature = []
train_loaders,_,_ = dataloader.get_dataloader(0)
for client in tqdm(range(args.n_clients)[:2]):
    tr_data = train_loaders[client]
    feature = []
    for image, _, _  in tr_data:
        image = image.to(args.device)
        image_feature = server_model.model.encode_image(image)
        for i in image_feature:
            feature.append(i.detach().cpu().numpy())
    all_feature.append(feature)


loading data in /mnt/sda/zd/data/splitdata_new/client0/task0/Art
loading data in /mnt/sda/zd/data/splitdata_new/client1/task0/Clipart
loading data in /mnt/sda/zd/data/splitdata_new/client2/task0/Clipart
loading data in /mnt/sda/zd/data/splitdata_new/client3/task0/Clipart
loading data in /mnt/sda/zd/data/splitdata_new/client4/task0/Clipart
loading data in /mnt/sda/zd/data/splitdata_new/client5/task0/Art
loading data in /mnt/sda/zd/data/splitdata_new/client6/task0/Product
loading data in /mnt/sda/zd/data/splitdata_new/client7/task0/Art
loading data in /mnt/sda/zd/data/splitdata_new/client8/task0/Product
loading data in /mnt/sda/zd/data/splitdata_new/client9/task0/Clipart
loading data in /mnt/sda/zd/data/splitdata_new/test/Clipart with ['Clipart', 'Product', 'Real World', 'Art']
loading data in /mnt/sda/zd/data/splitdata_new/test/Product with ['Clipart', 'Product', 'Real World', 'Art']
loading data in /mnt/sda/zd/data/splitdata_new/test/Real World with ['Clipart', 'Product', 'Real World',

  0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
t[0][0].shape

torch.Size([197, 100, 768])

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt

# 假设你有一个标签列表，其中每个标签对应一个feature对象
labels = list(itertools.chain.from_iterable([[i]*800 for i in range(10)]))# 这里的标签可以是任何你想要的标识符

# 将特征和标签合并成一个NumPy数组
all_feature = np.vstack(all_feature)  # 将所有特征堆叠成一个大的NumPy数组
labels = np.array(labels)  # 将标签转换为NumPy数组

print("执行t-SNE降维")
tsne = TSNE(n_components=2, perplexity=5, learning_rate=200, random_state=42)
tsne_result = tsne.fit_transform(all_feature)

print("可视化数据并使用标签着色")
plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=labels, cmap='viridis')  # 使用'viridis'颜色地图
plt.title("t-SNE Visualization with Labels")
plt.colorbar()
plt.show()

In [ ]:
print("执行t-SNE降维")
tsne = TSNE(n_components=2, perplexity=200, learning_rate=400, random_state=42)
tsne_result = tsne.fit_transform(all_feature)

print("可视化数据并使用标签着色")
plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=labels, s=3)  # 使用'viridis'颜色地图
plt.title("t-SNE Visualization with Labels")
plt.colorbar()
plt.show()

In [ ]:


import sys
import os
import itertools

base_path = os.path.dirname(os.path.dirname(os.path.abspath(".")))
sys.path.append(base_path)

import torch.optim as optim
import numpy as np
from tqdm import tqdm
import logging

from utils.config import img_param_init, set_random_seed
from utils.dataload import DomainDataset, get_data
# from model.clip import ClipModelMA, Adapter
import clip
from process import train_client, communicate, test_server, test_client, fetch

class argparser():
   device = "cuda:1"
   dataset = 'officehome'
   root_dir='/mnt/sda/zd/data/splitdata/'
   net='ViT-B/16'
   seed=0
   logdir=None
   device="cuda:1"
   n_clients=10
   n_experts=5
   inner_iter=10
   n_task=10
   # data setting parameter
   datapercent=8e-1
   batch=100
   # optimizer parameter
   lr=5e-3
   beta1=0.9
   beta2=0.98
   eps=1e-6
   weight_decay=0.2
    
def init():
    args = argparser()
    if args.logdir is not None:
        logging.basicConfig(filename=args.logdir, level=logging.DEBUG)
    else:
        logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
    args.random_state = np.random.RandomState(1)
    set_random_seed(args.seed)
    args = img_param_init(args)  # init the dataset parameters(domains and classnum)
    args.thresh = 1e-4
    return args

In [ ]:
args = init()
model, preprocess = clip.load("ViT-B/32")
model.to(args.device).eval()
dataloader: DomainDataset = get_data('officehome')(args, preprocess)

In [ ]:

all_feature = []
all_label = []
train_loaders,_,_ = dataloader.get_dataloader(0)
for client in tqdm(range(args.n_clients)):
    tr_data = train_loaders[client]
    feature = []
    label = []
    for image, _, labels  in tr_data:
        image, labels = image.to(args.device), labels.to(args.device)
        image_feature, f1= model.encode_image(image)
        for index, i in enumerate(f1):
            feature.append(i)
            label.append(labels[index].detach().cpu().numpy())
    all_feature.append(feature)
    all_label.append(label)
    

In [ ]:
import copy
saved_feature = copy.deepcopy(all_feature)
saved_label = copy.deepcopy(all_label)

In [ ]:
saved_label[0][:10]

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt

# 假设你有一个标签列表，其中每个标签对应一个feature对象
# labels = list(itertools.chain.from_iterable([[i]*800 for i in range(10)]))# 这里的标签可以是任何你想要的标识符
# labels = np.array(labels)  # 将标签转换为NumPy数组
labels = all_label
# 将特征和标签合并成一个NumPy数组
all_feature = np.vstack(all_feature)  # 将所有特征堆叠成一个大的NumPy数组
labels = np.vstack(labels)




In [ ]:
print("执行t-SNE降维")
tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, random_state=42)
tsne_result = tsne.fit_transform(all_feature[:,0,:])

print("可视化数据并使用标签着色")
plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=labels, cmap='viridis', s=3)  # 使用'viridis'颜色地图
plt.title("t-SNE Visualization with Labels")
plt.colorbar()
plt.show()

In [ ]:
# labels = list(itertools.chain.from_iterable([[i]*800 for i in range(10)]))# 这里的标签可以是任何你想要的标识符
labels = list(itertools.chain.from_iterable([[i]*800 for i in [0, 1, 0, 2, 0, 2, 0, 3, 1, 2]]))
labels = np.array(labels)  # 将标签转换为NumPy数组
result_list = []
for i in range(50):
    print("执行t-SNE降维")
    tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, random_state=42)
    tsne_result = tsne.fit_transform(all_feature[:,i,:])
    result_list.append(tsne_result)
    print("可视化数据并使用标签着色")
    plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=labels, cmap='viridis', s=3)  # 使用'viridis'颜色地图
    plt.title("t-SNE Visualization with Labels")
    plt.colorbar()
    plt.show()

In [ ]:

INFO:root:loading data in /mnt/sda/zd/data/splitdata/client0/task0/Art
INFO:root:loading data in /mnt/sda/zd/data/splitdata/client1/task0/Real World
INFO:root:loading data in /mnt/sda/zd/data/splitdata/client2/task0/Art
INFO:root:loading data in /mnt/sda/zd/data/splitdata/client3/task0/Clipart
INFO:root:loading data in /mnt/sda/zd/data/splitdata/client4/task0/Art
INFO:root:loading data in /mnt/sda/zd/data/splitdata/client5/task0/Clipart
INFO:root:loading data in /mnt/sda/zd/data/splitdata/client6/task0/Art
INFO:root:loading data in /mnt/sda/zd/data/splitdata/client7/task0/Product
INFO:root:loading data in /mnt/sda/zd/data/splitdata/client8/task0/Real World
INFO:root:loading data in /mnt/sda/zd/data/splitdata/client9/task0/Clipart